In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# !nvidia-smi
# model_0.summary()

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
!ls ./datasets/IDRiD/images/train|wc
# !ls ./

    413     413    5782


In [5]:
# !pip uninstall numpy
# !pip install numpy

In [6]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import PIL
from PIL import ImageFilter
# from tensorflow.python import keras
from tensorflow import keras
from tensorflow.keras.models import load_model, Model, save_model
# import tensorflow_hub as hub
# from tensorflow.keras.applications import EfficientNetB4, MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Average
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import *

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
import os

2023-07-10 20:04:40.729312: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [7]:
# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')
# mixed_precision.set_global_policy('float32')
import warnings
warnings.filterwarnings('ignore')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080 Ti, compute capability 7.5


2023-07-10 20:04:41.989259: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-10 20:04:41.990200: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-10 20:04:42.059199: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2023-07-10 20:04:42.059239: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-10 20:04:42.060796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-07-10 20:04:42.060853: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

In [8]:
# df=pd.read_csv("./datasets/Kaggle/retinopathy_solution.csv")
# df2=pd.read_csv("./datasets/Kaggle/trainLabels.csv")
# df=pd.concat([df, df2])
# df=df[['image','level']]
# df= df.drop_duplicates()
# df.to_csv('./datasets/Kaggle/all_labels.csv',header=True, index=False)

dataset_path = r'./datasets/'

idrid_path = dataset_path + "IDRiD/"
kaggle_path = dataset_path + "Kaggle/"
messidor_path = dataset_path + "Messidor-2/"

idrid_path_train_test = dataset_path + "IDRiD/images/train/"
kaggle_path_train_test = dataset_path + "Kaggle/images/"
messidor_path_train_test = dataset_path + "Messidor-2/images/"

kaggle_csv = "./datasets/Kaggle/all_labels.csv"
# kaggle_csv = "./datasets/Kaggle/trainLabels.csv"
idrid_csv = idrid_path + "a. IDRiD_Disease Grading_Training Labels.csv"
messidor_csv = messidor_path + "reference.csv"

idrid_columns = ['Image name', 'Retinopathy grade']
kaggle_columns = ['image', 'level']
messidor_columns = ['i_image','i_adjudicated_dr_grade']

In [9]:
# folder = r'./datasets/Messidor-2/images/'
# files = os.listdir(folder)
# files = os.listdir(idrid_path)

In [10]:
# Define the paths to the image directory and the CSV file
train_image_dir: str = idrid_path_train_test
# train_image_dir: str = kaggle_path_train_test
# train_image_dir: str = messidor_path_train_test
# csv_file_train = idrid_path + 'a. IDRiD_Disease Grading_Training Labels.csv'
# csv_file_test = idrid_path + 'b. IDRiD_Disease Grading_Testing Labels.csv'
csv_file = idrid_csv #kaggle_csv #messidor_csv
# Loading the dataset
df = pd.read_csv(csv_file)

# #uncomment for idrid data
df = df[idrid_columns]
print(df.shape)
df['Image name'] = df['Image name'].astype(str) + ".png"
df['Retinopathy grade'] = df['Retinopathy grade'].astype(str)
df.rename(columns={'Image name': 'img_name', 'Retinopathy grade': 'label'}, inplace=True)
df.dropna(how='all', inplace = True)
df = df[df['label'].notnull()]
df_shuffled = df.sample(frac=1, random_state=1)
df2 = df_shuffled.copy()
print(df2.shape)

# # #uncomment for kaggle data
# df = df[kaggle_columns]
# df['image'] = df['image'].astype(str) + ".png"
# df['level'] = df['level'].astype(str)
# df.rename(columns={'image': 'img_name', 'level': 'label'}, inplace=True)
# df.dropna(how='all', inplace = True)
# df = df[df['label'].notnull()]
# df_shuffled = df.sample(frac=1, random_state=1)
# df2=df_shuffled.iloc[:2048]
# df1=df_shuffled.iloc[2048:50048] #including 6000*4 number of elements i.e. max 6000 batches of 4 of which 4500 for training and 1500 for validation
# print(df1.head(),df1.shape, df2.head(),df2.shape)


##uncomment for messidor data
# df = df[messidor_columns]
# df['image'] = df['image'].astype(str) + ".png"
# df['level'] = df['level'].astype(str)
# df.rename(columns={'i_image': 'img_name', 'i_adjudicated_dr_grade': 'label'}, inplace=True)
# df.dropna(how='all', inplace = True)
# df = df[df['label'].notnull()]
# df_shuffled = df.sample(frac=1, random_state=1)
# df2 = df_shuffled.copy()
# print(df2.shape)

# Creating the image generator
# datagen = ImageDataGenerator(rescale=1. / 255)
#rotation_range --> minus to plus number of degrees of rotation
'''
ONE POSSIBLE PREPROCESSING FUNC preprocessing_function=lambda x: np.array(PIL.Image.fromarray(x).filter(ImageFilter.GaussianBlur(radius=2)).getdata()).reshape(data.size[::-1]+(-1,)).astype(np.uint8) if (np.random.rand() < 0.33) else np.array(PIL.Image.fromarray(x).filter(ImageFilter.SHARPEN).getdata()).reshape(data.size[::-1]+(-1,)).astype(np.uint8) if (np.random.rand() < 0.67) else x,
'''
datagen = ImageDataGenerator(rescale=None, rotation_range=5, zoom_range = 0.05, brightness_range =[0.8,1.2],
                             width_shift_range=15, height_shift_range=15, channel_shift_range = 10.0,
#                              preprocessing_function = basic_preprocessing_func,
                             horizontal_flip=True, validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=None)

# Creating the training set
# train_generator = datagen.flow_from_dataframe(
#     dataframe=df1,
#     directory=train_image_dir,
#     x_col="img_name",
#     y_col="label",
#     target_size=(512, 512),
#     batch_size=16,
#     class_mode='categorical',
#     subset='training'
# )

# validation_generator = datagen.flow_from_dataframe(
#     dataframe=df1,
#     directory=train_image_dir,
#     x_col="img_name",
#     y_col="label",
#     target_size=(512, 512),
#     batch_size=16,
#     class_mode='categorical',
#     subset='validation'
# )
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df2,
    directory=train_image_dir,
    x_col="img_name",
    y_col="label",
    target_size=(512, 512),
    batch_size=16,
    shuffle=False,
    class_mode='categorical'
    )

(413, 2)
(413, 2)
Found 413 validated image filenames belonging to 5 classes.


In [11]:
def aggregate_labels(y_true,y_pred):
    original_labels = [0, 1, 2, 3, 4] # original labels
    new_labels = [0, 1] # new labels
    label_map = {0: 0, 1: 0, 2: 1, 3: 1, 4: 1} # mapping from original labels to new labels
    new_y_true = np.zeros((y_true.shape[0], len(new_labels)))
    new_y_pred = np.zeros((y_pred.shape[0], len(new_labels)))
    for i, row in enumerate(y_true):
        for j, val in enumerate(row):
            if val == 1:
                new_label = label_map[original_labels[j]]
                new_y_true[i, new_label] = 1           
    for i, row in enumerate(y_pred):
        for j, val in enumerate(row):
            new_label = label_map[original_labels[j]]
            new_y_pred[i, new_label] += val
#     print("original predictions:\n", y_pred)        
#     print("Aggregated predictions:\n", new_y_pred)
    return new_y_true, new_y_pred

In [12]:
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, cohen_kappa_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
def plot_model(model_name,y_true_original,y_pred_original):
#     y_true = tf.keras.utils.to_categorical(test_generator.classes)
#     y_pred = model.predict_generator(test_generator)
    y_true,y_pred=aggregate_labels(y_true_original,y_pred_original)
    print(np.shape(y_true),np.shape(y_pred))
    # Compute ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(2):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
#         print(type(fpr[i]),len(fpr[i]),fpr[i])
#         print(np.shape(roc_auc[i]))
        print(roc_auc[i])
        print('_______________________')
#         print(type(tpr[i]),len(tpr[i]),tpr[i])
    print(len(y_true.ravel()))
    # Compute micro-average ROC curve and AUC
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Compute macro-average ROC curve and AUC
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(2)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(2):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= 2
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    aurocs = roc_auc_score(y_true, y_pred, multi_class = 'ovr', average = 'macro')
    print("AUROC score for model "+str(model_name)+":", aurocs)
    # Plot ROC curves for each class and micro/macro-average
    plt.figure()
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"], color='deeppink',
             lw=lw, label='micro-average ROC curve (AUC = %0.2f)' % roc_auc["micro"])
    plt.plot(fpr["macro"], tpr["macro"], color='navy',
             lw=lw, label='macro-average ROC curve (AUC = %0.2f)' % roc_auc["macro"])
    colors = ['blue', 'green', 'red', 'cyan', 'magenta']
    for i, color in zip(range(2), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class %d (AUC = %0.2f)' % (i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()
    
def plot_model_binary(model_name,y_true_original,y_pred_original):
    
    y_true,y_pred=aggregate_labels(y_true_original,y_pred_original)
#     print(np.shape(y_true),np.shape(y_pred))
    
    #saving data to verify manually
    l1 = list(y_true[:,0].T)
    l2 = list(y_true[:,1].T)
    l3 = list(y_pred[:,0].T)
    l4 = list(y_pred[:,1].T)
    l5 = list(np.where(y_pred[:,1].T>0.5,1,0))
    dictionary = {'True 0':l1,'True 1':l2,'Predicted 0':l3,'Predicted 1':l4,'Predicted_Onehot':l5}
    pd.DataFrame(dictionary).to_csv('results.csv', index = False, header = True)
    
    acc_score = accuracy_score(y_true[:,1],np.where(y_pred[:,1]<=0.5,0,1))
    print('Accuracy:',acc_score)
    
    kappa_score = cohen_kappa_score(np.argmax(y_true_original,axis=1 ), np.argmax(y_pred_original,axis=1), weights='quadratic')
    print('Quadratic Weighted Kappa:', kappa_score)
    
    # Compute ROC curve and AUC for each class
    # calculate the false positive rate, true positive rate and thresholds using roc_curve
    fpr, tpr, _ = roc_curve(y_true[:, 1], y_pred[:, 1])
    # calculate the AUC score using roc_auc_score
    auc_score = roc_auc_score(y_true[:, 1], y_pred[:, 1])
    print("AUROC score for model "+str(model_name)+":", auc_score)

    # plot the ROC curve
    plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.3f})'.format(auc_score))
    plt.plot([0, 1], [0, 1], linestyle='--', label='Random Guess')
    plt.xlabel('FPR ( or 1 minus Specificity)')
    plt.ylabel('TPR (or Sensitivity)')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
def plot_conf_matrix(model_name, y_true_original,y_pred_original):
    
    y_true,y_pred=aggregate_labels(y_true_original,y_pred_original)
    cm = confusion_matrix(y_true[:,1], np.where(y_pred[:,1]<=0.5,0,1))
    # Plot the confusion matrix
    plt.matshow(cm, cmap=plt.cm.Blues)
    plt.colorbar()
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(i, j, str(cm[i, j]), horizontalalignment='center', verticalalignment='center')
    plt.xlabel('True label')
    plt.ylabel('Predicted label')
    plt.title('Confusion Matrix for model:'+model_name)
    plt.show()

In [13]:
# plot_model_binary(models[0],y_true,y_pred)
# plot_conf_matrix(models[0],y_true,y_pred)

In [14]:
# models = ['MobNetv3small_0.75_224','EffNet_B4_imagenet']#
models = ['EffNet_B4_imagenet','EffNet_B5_imagenet','EffNet_B6_imagenet','EffNet_B7_imagenet','MobNetv3small_0.75_224','MobNetv3small_0.75_classbalanced','MobNetv3small_1.0','EffNet_B5_imagenet_regularized_highdropout','EffNet_B5_imagenet_norescale_sameastfmodel','EffNet_B6_imagenet_norescale_sameastfmodel','EffNet_B7_imagenet_norescale_sameastfmodel','EffNet_B4_imagenet_norescale_sameastfmodel','EffNet_B4_imagenet_classbalanced']
# for model in models[8:12]:
#     m = load_model('./'+model+'.h5')
#     y_true_model = tf.keras.utils.to_categorical(test_generator.classes)
#     y_pred_model = m.predict_generator(test_generator)
#     plot_model_binary(model,y_true_model,y_pred_model)
#     plot_conf_matrix(model,y_true_model,y_pred_model)

In [15]:
# type(test_generator.classes)
# y_pred_now[1]
# y_true_now[0]
# np.argmax(y_pred_now[0])

In [16]:
# y_pred_model
# x = load_model('./'+models[11]+'.h5')
# y_pred_now = x.predict(test_generator)
# y_true_now = tf.keras.utils.to_categorical(test_generator.classes)
# x = m.predict(test_generator)
# y_true_model.shape
# np.unique(y_true_model,return_counts=True)
# plot_conf_matrix(str(models[4:5]),y_true_model,y_pred_model)

In [17]:
# np.unique(x,return_counts=True)
# models[11]

In [18]:
# #Ensemble Strategy 1: Average
# def average_ensemble(models, generator):
#     # Make predictions on each model
#     y_preds=[]
#     for model in models:
#         y_preds.append(model.predict_generator(generator))
#     # Average the predictions across all models
# #     y_ensemble = np.nanmean(np.round(np.floor(y_preds*1000000)/1000000,3), axis=0)
# #     # Convert the averaged predictions to class labels
# #     y_ensemble = np.argmax(y_preds_avg, axis=1)
# #     y_ensemble=y_preds_avg
#     return y_preds

# #Ensemble Strategy 2: Majority Vote
# def majority_vote_ensemble(models, X):
#     # Make predictions on each model
#     y_preds = [model.predict(X) for model in models]
#     # Get the class with the highest probability from each model
#     y_preds_argmax = np.argmax(y_preds, axis=2)
#     # Compute the mode of the class labels across all models
#     y_ensemble = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=y_preds_argmax)
#     return y_ensemble

# #Ensemble Strategy 3: Max confidence
# def max_confidence_ensemble(models, X):
#     # Make predictions on each model
#     y_preds = [model.predict(X) for model in models]
#     # Get the maximum probability of each class from each model
#     y_preds_max = np.max(y_preds, axis=2)
#     # Compute the maximum probability across all models for each class
#     y_ensemble = np.argmax(np.sum(y_preds_max, axis=0))
#     return y_ensemble


In [19]:
# models = ['EffNet_B4_imagenet','EffNet_B5_imagenet','EffNet_B6_imagenet',
#           'EffNet_B7_imagenet','MobNetv3small_0.75_224','MobNetv3small_0.75_classbalanced',
#           'MobNetv3small_1.0','EffNet_B5_imagenet_regularized_highdropout','EffNet_B5_imagenet_norescale_sameastfmodel',
#           'EffNet_B6_imagenet_norescale_sameastfmodel','EffNet_B7_imagenet_norescale_sameastfmodel',
#           'EffNet_B4_imagenet_norescale_sameastfmodel','EffNet_B4_imagenet_classbalanced']
# loaded_models=[]
# for i in range(len(models[8:12])):
#     var_name = f"model_{i}"
# #     var_value = models[i]
#     var_value = load_model('./'+models[i]+'.h5')
#     globals()[var_name] = var_value
#     loaded_models.append(globals()[var_name])
# # y_pred_ensemble = average_ensemble(loaded_models,test_generator)
# # plot_model_binary(model,y_true_ensemble,y_pred_ensemble)
# # plot_conf_matrix(model,y_true_ensemble,y_pred_ensemble)

In [20]:
models = ['MobNetv3small_1.0_rev','EffNet_B4_imagenet_norescale_sameastfmodel_rev','EffNet_B5_imagenet_norescale_sameastfmodel_rev','EffNet_B6_imagenet_norescale_sameastfmodel_rev','EffNet_B7_imagenet_norescale_sameastfmodel_rev']
models[1:]

['EffNet_B4_imagenet_norescale_sameastfmodel_rev',
 'EffNet_B5_imagenet_norescale_sameastfmodel_rev',
 'EffNet_B6_imagenet_norescale_sameastfmodel_rev',
 'EffNet_B7_imagenet_norescale_sameastfmodel_rev']

### TRYING TO SAVE AN ENSEMBLE OUT OF INDIVIDUAL MODELS

In [9]:
# models = ['MobNetv3small_1.0_rev','EffNet_B4_imagenet_norescale_sameastfmodel_rev','EffNet_B5_imagenet_norescale_sameastfmodel_rev','EffNet_B6_imagenet_norescale_sameastfmodel_rev','EffNet_B7_imagenet_norescale_sameastfmodel_rev']
# mods=['EffNet_B5_imagenet_norescale_sameastfmodel','EffNet_B6_imagenet_norescale_sameastfmodel',
#  'EffNet_B7_imagenet_norescale_sameastfmodel','EffNet_B4_imagenet_norescale_sameastfmodel']

# mods = ['EffNet_B4_exp14_NoActivation', 'EffNet_B5_exp15_NoActivation', 'EffNet_B6_exp16_NoActivation']
# mods = ['EffNet_B0_imagenet_norescale_sameastfmodel_rev_NoActivation', 'EffNet_B1_imagenet_norescale_sameastfmodel_rev_NoActivation','EffNet_B2_imagenet_norescale_sameastfmodel_rev_NoActivation']
# mods = ['EffNetB0_distilled_finalexp21', 'EffNetB1_distilled_finalexp25','EffNetB2_distilled_finalexp26']
# mods = models[1:-1]
mods=['EffNet_B5_imagenet_Glaucoma','EffNet_B6_imagenet_Glaucoma','EffNet_B7_imagenet_Glaucoma']
# Load the individual models and change the names of the layers to be unique
model1 = load_model('./models/teacher_experiments/Glaucoma/'+mods[0]+'.h5', compile=False, custom_objects={'tf': tf})
model1 = Model(inputs=model1.input, outputs=model1.output, name=mods[0])
model2 = load_model('./models/teacher_experiments/Glaucoma/'+mods[1]+'.h5', compile=False, custom_objects={'tf': tf})
model2 = Model(inputs=model2.input, outputs=model2.output, name=mods[1])
model3 = load_model('./models/teacher_experiments/Glaucoma/'+mods[2]+'.h5', compile=False, custom_objects={'tf': tf})
model3 = Model(inputs=model3.input, outputs=model3.output, name=mods[2])
# model4 = load_model('./'+mods[3]+'.h5', compile=False, custom_objects={'tf': tf})
# model4 = Model(inputs=model4.input, outputs=model4.output, name=mods[3])

# Define the input tensor
inp = Input(shape=(512, 512, 3), name = 'input')
# Get the outputs of the individual models
model1_output = model1(inp)
model2_output = model2(inp)
model3_output = model3(inp)
# model4_output = model4(inp)
# Average the outputs of the two models
ensemble_output = Average(name = 'average')([model1_output, model2_output, model3_output])#, model4_output])

# Define the ensemble model
# ensemble_model = Model(inputs=inp, outputs=ensemble_output, name = 'ensemble_b0b1_scratch_logits')
# ensemble_model = Model(inputs=inp, outputs=ensemble_output, name = 'ensemble_b0b1b2_scratch_logits')
ensemble_model = Model(inputs=inp, outputs=ensemble_output, name = 'ensemble_b5b6b7_glaucoma')

# Save the ensemble model
# save_model(ensemble_model, 'teacher_ensemble_NoActivation.h5')
# save_model(ensemble_model, 'student_scratch_ensemble_b0b1_NoActivation.h5')
# save_model(ensemble_model, './models/student_experiments/student_scratch_ensemble_b0b1b2_NoActivation.h5')
save_model(ensemble_model, './models/teacher_experiments/Glaucoma/ensemble_b5b6b7.h5')

# Load the ensemble model for prediction later
# loaded_ensemble_model = load_model('teacher_ensemble_b4b5b6.h5')
# loaded_ensemble_model = ensemble_model
# Use the loaded ensemble model to make predictions
# y_true_ensemble = tf.keras.utils.to_categorical(test_generator.classes)
# y_pred_ensemble = loaded_ensemble_model.predict_generator(test_generator)
# plot_model_binary('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)
# plot_conf_matrix('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)

2023-07-10 20:13:24.878379: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 20:13:24.881916: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-10 20:13:24.882629: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2023-07-10 20:13:24.882671: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-10 20:13:24.882692: I tensorflow/stream_executor/

In [10]:
ensemble_model.summary()

Model: "ensemble_b5b6b7_glaucoma"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
EffNet_B5_imagenet_Glaucoma (Fu (None, 2)            28517625    input[0][0]                      
__________________________________________________________________________________________________
EffNet_B6_imagenet_Glaucoma (Fu (None, 2)            40964753    input[0][0]                      
__________________________________________________________________________________________________
EffNet_B7_imagenet_Glaucoma (Fu (None, 2)            64102809    input[0][0]                      
___________________________________________________________________________

In [ ]:
# plot_model_binary('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)
# plot_conf_matrix('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)

### Calculating Ensemble Performance Here

In [ ]:
mods=['EffNet_B5_imagenet_norescale_sameastfmodel','EffNet_B6_imagenet_norescale_sameastfmodel',
 'EffNet_B7_imagenet_norescale_sameastfmodel','EffNet_B4_imagenet_norescale_sameastfmodel']
model1 = load_model('./'+mods[0]+'.h5')
model2 = load_model('./'+mods[1]+'.h5')
model3 = load_model('./'+mods[2]+'.h5')
model4 = load_model('./'+mods[3]+'.h5')
y_true_ensemble = tf.keras.utils.to_categorical(test_generator.classes)
all_models=[model1,model2,model3,model4]
y_pred_ensemble=[]
for model in all_models:
        y_pred_ensemble.append(model.predict_generator(test_generator))

In [ ]:
y_pred_ensemble = np.mean(y_pred_ensemble, axis=0)
y_true_ensemble = tf.keras.utils.to_categorical(test_generator.classes)
plot_model_binary('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)
plot_conf_matrix('Teacher_ensemble',y_true_ensemble,y_pred_ensemble)

In [ ]:
modelmobilenet = MobileNetV3Small(input_shape=(512, 512, 3),include_top=False,weights='imagenet',pooling='avg',alpha=1.0)
modelmobilenet.summary()